In [1]:
import csv
import networkx as nx
import numpy as np
import random

In [2]:
G = nx.read_edgelist('../data/initial_data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

Number of nodes: 138499
Number of edges: 1091955


In [3]:
node_pairs_test = list()
with open('../data/initial_data/test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs_test.append((int(t[0]), int(t[1])))

node_pairs_test = list(map(sorted, node_pairs_test))
# The result of sorted is a list so we put it back as a tuple
node_pairs_test = list(map(tuple, node_pairs_test))

In [4]:
node_pairs_train = list(G.edges())

node_pairs_train = list(map(sorted, node_pairs_train))
# The result of sorted is a list so we put it back as a tuple
node_pairs_train = list(map(tuple, node_pairs_train))

In [5]:
len(set(node_pairs_train) & set(node_pairs_test))

0

We can see that none of the node pairs of the test set are not edges in the graph. We don't know if there is a citation link between each node pair of the test. 

In [27]:
random.seed(12)
non_edge_pairs = [(random.randint(0, n-1), random.randint(0, n-1)) for _ in range(m+1000)]

In [28]:
non_edge_pairs = list(map(sorted, non_edge_pairs))
# The result of sorted is a list so we put it back as a tuple
non_edge_pairs = list(map(tuple, non_edge_pairs))

In [32]:
non_edge_pairs = set(non_edge_pairs) - (set(node_pairs_test) | set(node_pairs_train))
non_edge_pairs = list(non_edge_pairs)[:m]

In [34]:
assert(len(non_edge_pairs) == len(node_pairs_train))

In [35]:
with open("../data/processed_data/non_edges.txt", 'w+') as f:
    for node1, node2 in non_edge_pairs:
        f.write(f"{node1},{node2}\n")